In [ ]:
import histdatacom
from histdatacom.options import Options
options = Options()

options.api_return_type = "pandas"  # "datatable", "pandas", or "arrow"
options.formats = {"ascii"}  # Must be {"ascii"}
options.timeframes = {"tick-data-quotes"}  # can be tick-data-quotes or 1-minute-bar-quotes
options.pairs = {"eurusd"}
options.start_yearmonth = "2020-01"
options.end_yearmonth = "now"
options.cpu_utilization = "high"

data = histdatacom(options)  # (Jupyter)

In [ ]:
data_list = data.copy()

In [ ]:
print(data_list)

In [ ]:
# DT = data.copy()
# print(f"""
#     type: {type(DT)}
#     shape: {DT.shape}
#     names: {DT.names}
#     types: {DT.stypes}\n\n""",
#     DT[0:5,:])


In [ ]:
# DT = data.copy()
# dt_timestamp_to_datetime = ((f.datetime * 10**6).as_type(dt.Type.time64))
# DT[:, update(datetime = dt_timestamp_to_datetime)]
# print(f"""
#     type: {type(DT)}
#     shape: {DT.shape}
#     names: {DT.names}
#     types: {DT.stypes}\n\n""",
#     DT[0:5,:])

In [ ]:
options.api_return_type = "pandas"
options.formats = {"ascii"}
options.timeframes = {"1-minute-bar-quotes"}
options.pairs = {"eurusd","usdcad"}
options.start_yearmonth = "2021-01"
options.end_yearmonth = "now"
options.cpu_utilization = "75"